# DataAugmentation

Este notebook muestra el proceso de aumento de datos utilizando TensorFlow y la biblioteca ImageDataGenerator.

Se lee un archivo CSV que contiene la información de las imágenes y se seleccionan aleatoriamente un número determinado de imágenes por clase. A continuación, se aplica el aumento de datos utilizando diferentes transformaciones, como rotación, desplazamiento, corte, zoom y volteo horizontal. Las imágenes originales y aumentadas se visualizan en un conjunto de subplots para su comparación.

Este proceso es útil para generar un conjunto de datos de entrenamiento más diverso y aumentar la cantidad de imágenes disponibles para el entrenamiento de modelos de aprendizaje automático, como redes neuronales convolucionales.

In [24]:
# Importar las bibliotecas necesarias
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [25]:
# Verificar si hay una GPU disponible
if tf.config.list_physical_devices('GPU'):
    print('TensorFlow está utilizando la GPU')

    # Crear un tensor en la GPU
    with tf.device('/GPU:0'):
        tensor = tf.constant([1, 2, 3])

    # Realizar alguna operación en la GPU
    result = tensor * 2

    # Imprimir el resultado
    print(result)
else:
    print('TensorFlow está utilizando la CPU')

TensorFlow está utilizando la GPU
tf.Tensor([2 4 6], shape=(3,), dtype=int32)


In [26]:
# Ruta del archivo CSV de entrada
csv_path = "../../CSVs/c01_TOY_reduced.csv"
csv_train = "../../CSVs/dataset_toy/train.csv"
csv_test = "../../CSVs/dataset_toy/test.csv"
csv_val = "../../CSVs/dataset_toy/val.csv"

# Ruta del directorio donde se encuentran las imágenes
dataset_path = "../../Dataset/"

### Lectura de los archivos CSVs

In [27]:
# Lectura de los archivos CSV
df_train = pd.read_csv(csv_train)
df_test = pd.read_csv(csv_test)
df_val = pd.read_csv(csv_val)

In [28]:
# Número de clases
num_classes = df_train['class'].nunique()

### Creación del modelo

#### Base

EfficientNet es una familia de redes neuronales convolucionales eficientes en términos de recursos computacionales. La variante EfficientNetB0 es la más pequeña de la familia, pero aún así logra un buen rendimiento en una amplia gama de tareas. Puedes utilizar el módulo tensorflow.keras.applications.efficientnet para cargar el modelo pre-entrenado y adaptarlo según tus requisitos.

In [39]:
# Configuración de la red EfficientNetB0
from tensorflow.keras.applications.efficientnet import EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#### Últimas capas

In [40]:
# Últimas capas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

#### Modelo Final

In [41]:
# Modelo Final
model = Model(inputs=base_model.input, outputs=predictions)

In [42]:
# Congelar los pesos de las capas base (excepto las últimas capas añadidas)
for layer in base_model.layers:
    layer.trainable = False

In [43]:
# Compilar el modelo
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [44]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_10[0][0]']               
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

### Aumento de datos

In [45]:
# Aumento de datos
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    # vertical_flip=True,
)

### Generador de conjuntos

In [46]:
# Crear generador de datos para el conjunto de entrenamiento
train_datagen = datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=dataset_path,
    x_col='path',
    y_col='class',
    target_size=(224, 224),
    batch_size=4,
    class_mode='categorical'
)

# Crear generador de datos para el conjunto de validación
valid_datagen = datagen.flow_from_dataframe(
    dataframe=df_val,
    directory=dataset_path,
    x_col='path',
    y_col='class',
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical'
)

# Crear generador de datos para el conjunto de prueba
test_datagen = datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=dataset_path,
    x_col='path',
    y_col='class',
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical'
)

Found 23815 validated image filenames belonging to 16 classes.
Found 2977 validated image filenames belonging to 16 classes.
Found 2977 validated image filenames belonging to 16 classes.


### Test sin entrenar con datos

In [47]:
# Evaluar el modelo en el conjunto de prueba
'''test_loss, test_accuracy = model.evaluate(test_datagen)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")'''

'test_loss, test_accuracy = model.evaluate(test_datagen)\nprint(f"Test Loss: {test_loss:.4f}")\nprint(f"Test Accuracy: {test_accuracy:.4f}")'

### Control de entrenamiento

In [48]:
# Estimación del error en el conjunto de validación
validation_steps = len(df_val) // 32

In [49]:
# Definir Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# Entrenar el modelo
history = model.fit(
    train_datagen,
    steps_per_epoch=len(df_train) // 8,
    validation_data=valid_datagen,
    validation_steps=validation_steps,
    epochs=10,
    callbacks=[early_stopping]
    #workers=1,
    #use_multiprocessing=False'''
)

Epoch 1/10


In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(test_datagen)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# Visualizar el entrenamiento
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()